In [1]:
##--- script to look for unsorted sessions for a given rat, copy them locally, sort, copy to cup, and remove --##

import os
import shutil
from math import ceil,floor
from pathlib import Path,WindowsPath
#from pykilosort import run, add_default_handler, np1_probe, np2_probe, np2_probe_sglx
from pykilosort import add_default_handler, np1_probe, np2_probe, np2_probe_sglx
from pykilosort.main import run
from DemoReadSGLXData.readSGLX import readMeta
import subprocess

# set up data paths
archive_header = Path('Y:')
local_header = Path('E:')
cup_header = Path('X:')
data_dir = Path('RATTER\\PhysData\\Raw\\SarahJo')
sorted_dir = Path('RATTER\\PhysData\\NP_sorted\\SarahJo')

# select rat
rat_id = 'M136'
archive_dir = Path(archive_header,data_dir,rat_id)
local_dir = Path(local_header,data_dir,rat_id)
day_fldrs = os.listdir(archive_dir)

In [ ]:
# for each day
for day_fldr in day_fldrs[7:]:
    day_path = Path(archive_dir,day_fldr)
    session_fldrs = os.listdir(day_path)
    # for each recorded session
    for session_fldr in session_fldrs:
        session_path = Path(day_path,session_fldr)
        if os.path.isdir(session_path):
            probe_fldrs = os.listdir(session_path)
            # for each probe
            for probe_fldr in probe_fldrs:
                probe_path = Path(session_path,probe_fldr)
                if os.path.isdir(probe_path):
                    # check if sorted files exist on cup
                    sorted_path = Path(cup_header,sorted_dir,*probe_path.parts[5:],'output') 
                    if os.path.isdir(sorted_path)!=True:
                        print('---------- sorting probe',probe_fldr,'----------')
                        probe_files = os.listdir(probe_path)
                        local_path = Path(local_header,*probe_path.parts[1:])

                        # first file is .bin file
                        data_path = Path(local_path,probe_files[0])
                        # second file is .meta file
                        meta_path = Path(local_path,probe_files[1])

                        # copy data from archive if necessary
                        if os.path.isfile(data_path)!=True or os.path.isfile(meta_path)!=True:
                            print('-'*floor((len(probe_fldr)+9)/2),'copying data from archive','-'*ceil((len(probe_fldr)+9)/2))
                            try:
                                shutil.copytree(probe_path,local_path)
                            except:
                                # in case of a previous partial transfer, delete old files
                                shutil.rmtree(local_path)
                                shutil.copytree(probe_path,local_path)

                        # save initial sort locally
                        output_path = Path(local_header,*sorted_path.parts[1:-1])

                        # get metadata 
                        meta = readMeta(meta_path)

                        # run pykilosort
                        print('-'*floor((len(probe_fldr)+17)/2),'starting kilosort','-'*ceil((len(probe_fldr)+17)/2))
                        try:
                            # can't delete local raw data when I do this
                            #add_default_handler(level='INFO')
                            #run(data_path, dir_path=output_path, probe=np2_probe_sglx(meta_path), low_memory=True)

                            # can delete local raw data if I run in subprocess
                            subprocess.call(f'conda activate pyks2 && python run_pykilosort.py {data_path} {output_path} {meta_path}', shell=True)
                        except:
                            # in case of a previous partial run, delete old files
                            shutil.rmtree(output_path)
                            subprocess.call(f'conda activate pyks2 && python run_pykilosort.py {data_path} {output_path} {meta_path}', shell=True)


                        # copy output to cup
                        print('-'*floor((len(probe_fldr)+20)/2),'copying to cup','-'*ceil((len(probe_fldr)+20)/2))
                        shutil.copytree(Path(output_path,'output'),sorted_path)

                        # delete local files
                        print('-'*floor((len(probe_fldr)+14)/2),'removing local files','-'*ceil((len(probe_fldr)+14)/2))
                        shutil.rmtree(output_path)
                        shutil.rmtree(local_path)  # this errors out if pykilosort is run locally
                    else:
                        print('----------',probe_fldr,'already sorted ---------')

    
print('-'*floor((len(probe_fldr)+29)/2),'done!','-'*ceil((len(probe_fldr)+29)/2))

---------- sorting probe M136_2021_12_14_rig204_g0_imec0 ----------
-------------------- copying data from archive --------------------
------------------------ starting kilosort ------------------------
------------------------- copying to cup --------------------------
---------------------- removing local files -----------------------
---------- sorting probe M136_2021_12_14_rig204_g0_imec1 ----------
-------------------- copying data from archive --------------------


In [ ]:
from pathlib import Path,WindowsPath
from pykilosort import run, add_default_handler, np1_probe, np2_probe, np2_probe_sglx
from DemoReadSGLXData.readSGLX import readMeta


archive_path = Path('Y:\\brody\\RATTER\\PhysData\\Raw\\SarahJo')
rat_id = 'M136'



data_path1 =Path('E:\\RATTER\\PhysData\\Raw\\SarahJo\\M136\\M136_2021_12_08\\M136_2021_12_08_rig204_g0\\M136_2021_12_08_rig204_g0_imec1\\M136_2021_12_08_rig204_g0_t0.imec1.ap.bin')
#data_path2 = Path('E\\RATTER\\PhysData\\Raw\\SarahJo\\M136\\M136_2021_12_08\\M136_2021_12_08_rig204_g0\\M136_2021_12_08_rig204_g0_imec1\\M136_2021_12_08_rig204_g0_t0.imec1.ap.bin')
data_paths = [
    data_path1,
    #data_path2,
]
dir_path = Path('E:\\RATTER\\PhysData\\Raw\\SarahJo\\M136\\M136_2021_12_08\\M136_2021_12_08_rig204_g0\\M136_2021_12_08_rig204_g0_imec1')
meta_path = Path('E:\\RATTER\\PhysData\\Raw\\SarahJo\\M136\\M136_2021_12_08\\M136_2021_12_08_rig204_g0\\M136_2021_12_08_rig204_g0_imec1\\M136_2021_12_08_rig204_g0_t0.imec1.ap.meta')
meta = readMeta(meta_path)

add_default_handler(level='INFO')
run(data_paths, dir_path=dir_path, probe=np2_probe_sglx(meta_path), low_memory=True)

In [ ]:
import spikeinterface.extractors as se
from nwb_conversion_tools.tools.spikeinterface import write_recording, write_sorting
from datetime import datetime


phy_path = dir_path / "output"
sorting_curated = se.read_phy(phy_path, exclude_cluster_groups=['noise'])

nwbfile_path = dir_path / "sorting_curated.nwb"

some_datetime_object = datetime(2021, 12, 8)


write_sorting(
    sorting=sorting_curated.select_units(np.arange(0,507)),
    save_path=nwbfile_path,
    metadata = dict(NWBFile=dict(session_start_time=some_datetime_object)),
    overwrite=True,  # this appends the file. True would write a new file,
)

In [ ]:
import numpy as np
sorting_curated.select_units(np.arange(0,507))